# Dive into Python: GIL and GC

## Эксперимент с CPU-bound функцией

In [1]:
def factorize_naive(n):
    """ A naive factorization method. Take integer 'n', return list of
        factors.
    """
    if n < 2:
        return []
    factors = []
    p = 2

    while True:
        if n == 1:
            return factors

        r = n % p
        if r == 0:
            factors.append(p)
            n = n / p
        elif p * p >= n:
            factors.append(n)
            return factors
        elif p > 2:
            # Advance in steps of 2 over odd numbers
            p += 2
        else:
            # If p == 2, get to 3
            p += 1
    assert False, "unreachable"

In [2]:
def serial_factorizer(count):
    return {n: factorize_naive(n) for n in range(1, count + 1)}

In [3]:
%time results = serial_factorizer(2000000)

CPU times: user 1min 22s, sys: 168 ms, total: 1min 22s
Wall time: 1min 22s


In [4]:
import threading
import math

def threaded_factorizer(count, nthreads):
    def worker(nums, outdict):
        """ The worker function, invoked in a thread. 'nums' is a
            list of numbers to factor. The results are placed in
            outdict.
        """
        for n in nums:
            outdict[n] = factorize_naive(n)

    # Each thread will get 'chunksize' nums and its own output dict
    chunksize = int(math.ceil(count / float(nthreads)))
    threads = []
    outs = [{} for i in range(nthreads)]

    for i in range(nthreads):
        # Create each thread, passing it its chunk of numbers to factor
        # and output dict.
        t = threading.Thread(
                target=worker,
                args=(range(chunksize * i + 1, chunksize * (i + 1) + 1),
                      outs[i]))
        threads.append(t)
        t.start()

    # Wait for all threads to finish
    for t in threads:
        t.join()

    # Merge all partial output dicts into a single dict and return it
    return {k: v for out_d in outs for k, v in out_d.items()}

In [5]:
%time results = threaded_factorizer(2000000, 2)

CPU times: user 1min 24s, sys: 424 ms, total: 1min 25s
Wall time: 1min 25s


In [6]:
%time results = threaded_factorizer(2000000, 4)

CPU times: user 1min 27s, sys: 544 ms, total: 1min 28s
Wall time: 1min 28s


In [7]:
%time results = threaded_factorizer(2000000, 8)

CPU times: user 1min 23s, sys: 636 ms, total: 1min 24s
Wall time: 1min 23s


## Python threads

* Потоки в Python это реальные потоки в операционной системе
    * POSIX threads (pthreads)
    * Windows threads
* Управляются операционной системой
* Представляют собой исполнение потока выполнения кода в процессе интерпретатора (кода на C)

# GIL

* Параллельное исполнение потоков запрещено
* Используется сущность "global interpreter lock" (GIL) - "глобальный лок интерпретатора"
* GIL обеспечивает, что в любой момент времени есть не более одного исполняющегося потока
* Сильно упрощает код самого интерпретатора (управление памятью, вызов C-extensions и т.д.)

## Модель исполнения потоков

* Когда поток исполняется, он захватывает GIL
* GIL отпускает на I/O операциях (read/write/send/recv и т.д.)
* C GIL получается cooperative multitasking

<img src="https://i.imgur.com/EPXn5sz.png">

## Thread scheduling в "старом" GIL

Каждые N тиков (выполнения N инструкций виртуальной машины Python) каждый исполняющийся поток делал:
* Посылал сигнал спящим потокам
* Отпускал GIL
* Пытался получить GIL

<img src="https://i.imgur.com/GXU4eCX.png">


## "Драка" за GIL

* На многоядерных процессорах, CPU-bound потоки шедулятся ОС одновременно на разные ядра и начинают "драться за GIL"

<img src="https://i.imgur.com/GYD8V0q.png">

* Может занять пару сотен попыток для потока получить GIL

<img src="https://i.imgur.com/B8iTN92.png">

## "Новый" GIL (c версии 3.2)

* GIL никуда не исчез - была улучшена его реализация
* Новая реализация стремилась получить более консистентное поведения потоков в рантайме интерпретатора за счет сокращения накладных расходов таких "драк"

* Прошлая версия была основана на подсчете тиков, т.е. выполненных инструкций, и посылании сигналов на пробуждение спящим потокам каждые N тиков, чтобы у них появилась возможность захватить лок.
* В новой версии никаких тиков нет и применяется time-based подход.

* Решение провести switch исполняющегося потока привязано к глобальной переменной
```
/* Python/ceval.c */
...
static volatile int gil_drop_request = 0;
```
* Поток исполняется "вечно" до того момента, пока эта переменная не установлена в 1.
* В этот момент поток должен отпустить GIL.

# Случай одного потока

<img src="https://i.imgur.com/eW5Vayb.png">

Он исполняется вечно и никогда не отпускает GIL.

# Случай двух потоков

<img src="https://i.imgur.com/dPepqSZ.png">

* Легкий случай: второй поток будет ждать того момента, пока первый поток добровольно отпустит GIL, к примеру, если первому потоку нужно будет произвести I/O-операцию или "заснуть" на некоторое время.

<img src="https://i.imgur.com/ZBRxbwe.png">

* Сложный случай: поток 2 ждет некоторого таймаута, после которого установливает `gil_drop_request` в `1` и снова переходит в режим ожидания.
* В этом случае поток 1 вынужден отпустить GIL: он завершает выполнение последней инструкции, отпускает GIL и сигнализирует потоку 2, что он отпустил его.
* Поток 2, захватив GIL, оповещает об этом поток 1 и далее уже поток 1 переходит в режим ожидания.
* Никакой драки за GIL не происходит.

Эти действия повторяются все время исполнения процесса.

<img src="https://i.imgur.com/NBTBApi.png">

* Дефолтное значение таймаута 5 миллисекунд.
* Можно поменять с помощью `sys.setswitchinterval()`

## Случай многих потоков

* По истечении таймаута на получение GIL для некоторого потока `gil_drop_request` устанавливается лишь в том случае, если за этот таймаут не происходило никакого thread-switch каким-либо другим потоком.

<img src="https://i.imgur.com/1wRop1Y.png">

* GIL может получить не обязательно тот поток, который установил `gil_drop_request`. Во многом это зависит от планировщика ОС.

<img src="https://i.imgur.com/bD4Kgbp.png">

## Интересные свойства

* GIL позволяет одному потоку исполняться непрерывно на протяжении 5 мс в независимости от других потоков или приоритетов ОС.
* На это время CPU-bound может заблокировать IO-bound поток, что может ухудшить характеристики времен ответа для веб-серверов и т.д., т.е. в этом случае стоит "подкрутить" этот интервал.
* Вызов C/С++-extensions может заблокировать thread-switch на неопределенный период.
* thread-switсh не является preemptive

# Garbage collection

* Обычно программисту не нужно заботиться об управлении памятью в Python.
* Когда объекта становятся не нужны, Python автоматически освобождает память из-под них.
* Но есть особенности :)

## Memory management

* Python не сразу возвращает свободную память операционной системе.
* Он имеет специальный аллокатор для маленьких объектов (меньше 512 байт), который хранит "свободные", полученные от ОС, чанки памяти для использования в будущем.

СPython реализация garbage collector-а имеет два компонента:
* счетчик ссылок (reference counter)
* genarational cyclic garbage collector, известный как модуль `gc`

Счетчик ссылок очень простой и эффективный, но он не может удалять циклы ссылок, тогда вступает в игру generational cyclic gc.

# Reference counting

* Объекты удаляются, когда на них не остается ссылок.
* Каждый объект имеет дополнительное поле reference count, когда инкрементится или декрементится, когда ссылка на объет добавляется или удаляется.
* Когда счетчик становится равным нулю, объект автоматически удаляется. Если он содержит ссылки на другие объекты, то их счетчики декрементятся и если счетчик вложенного объекта становится равным нулю, то происходит уже его удаление. И так далее по цепочке.
* Переменные, объявленные глобально, живут до конца программы, если их не удалить вручную.
* Счетчики на переменные, объявленные в локальном скоупе, декрементятся, когда происходит выход из этого скоупа.

In [8]:
import sys

foo = []

# 2 references, 1 from the foo var and 1 from getrefcount
print(sys.getrefcount(foo))

def bar(a):
    # 4 references
    # from the foo var, function argument, getrefcount and Python's function stack
    print(sys.getrefcount(a))

bar(foo)
# 2 references, the function scope is destroyed
print(sys.getrefcount(foo))

2
4
2


## Generational garbage collector

У подхода, основанного на подсчете ссылок, есть один фундаментальный недостаток: неспособность обрабатывать циклические зависимости.

<img src="https://rushter.com/static/uploads/img/circularref.svg">

In [10]:
import gc
import ctypes

# We are using ctypes to access our unreachable objects by memory address.
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]


# gc.disable()  # Disable generational gc

lst = []
lst.append(lst)

# Store address of the list
lst_address = id(lst)

# Destroy the lst reference
del lst

object_1 = {}
object_2 = {}
object_1['obj2'] = object_2
object_2['obj1'] = object_1

obj_address = id(object_1)

# Destroy references
del object_1, object_2

# Uncomment if you want to manually run garbage collection process 
# gc.collect()

# Check the reference count
print(PyObject.from_address(obj_address).refcnt)
print(PyObject.from_address(lst_address).refcnt)

1
1


* Для решения проблемы циклических ссылок, был реализован специальный алгоритм, призванный детектировать циклические ссылки, доступный через модуль `gc`.

## Устройство generational garbage collector

* Не работает в realtime, в отличие от reference count, запускается периодически.
* Есть 3 поколения. Каждый новый объект-контейнер помещается в первое поколение. Если объект остаесть после какой-либо итерации `gc`, он двигается на поколение вверх, если возможно.
* Каждое поколение имеет свой счетчик и величину порога. В счетчике хранится разница между количеством аллокаций и деаллокаций с последней итерации `gc`. Если этот счетчик превысит определенный порог для первого поколения, запустится новая итерация.
* Если есть несколько поколений с превышенным порогам, то итерация `gc` запускается на более старшем.
* Стандартные пороги `(700, 10, 10)`, можно проверить через `gc.get_threshold`.

## Как находить циклы

* Идея: пытаемся для каждого объекта в поколении убрать ссылки на объекты, на которые он ссылается. Пройдя по всем объектам, удаляем те, на которые осталось меньше двух ссылок.
* Больше подробностей здесь https://pythoninternal.wordpress.com/2014/08/04/the-garbage-collector/

<img src="https://pythoninternal.files.wordpress.com/2014/07/python-cyclic-gc-1-new-page.png">
<img src="https://pythoninternal.files.wordpress.com/2014/07/python-cyclic-gc-2-new-page.png">
<img src="https://pythoninternal.files.wordpress.com/2014/07/python-cyclic-gc-3-new-page.png">
<img src="https://pythoninternal.files.wordpress.com/2014/07/python-cyclic-gc-4-new-page.png">
<img src="https://pythoninternal.files.wordpress.com/2014/07/python-cyclic-gc-5-new-page.png">

## Советы по оптимизации

* Использовать `weakref` модуль. `weakref.ref` не инкрементит счетчик ссылок на объект и возвращает `None`, если объект удалили.
* `gc.disable()` и `gc.collect()` для ручного управления.

## Дебаг циклических ссылок

In [ ]:
import gc

gc.set_debug(gc.DEBUG_SAVEALL)

print(gc.get_count())
lst = []
lst.append(lst)
print(id(lst))
del lst
gc.collect()
for item in gc.garbage:
    print(id(item))